In [1]:
!nvidia-smi

Sun Feb  8 08:29:27 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   50C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip uninstall -y torch torchvision torchaudio mamba-ssm causal-conv1d

!pip install torch==2.7.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

!pip install https://github.com/Dao-AILab/causal-conv1d/releases/download/v1.6.0/causal_conv1d-1.6.0+cu12torch2.7cxx11abiFALSE-cp312-cp312-linux_x86_64.whl

Found existing installation: torch 2.9.0+cu126
Uninstalling torch-2.9.0+cu126:
  Successfully uninstalled torch-2.9.0+cu126
Found existing installation: torchvision 0.24.0+cu126
Uninstalling torchvision-0.24.0+cu126:
  Successfully uninstalled torchvision-0.24.0+cu126
Found existing installation: torchaudio 2.9.0+cu126
Uninstalling torchaudio-2.9.0+cu126:
  Successfully uninstalled torchaudio-2.9.0+cu126
Looking in indexes: https://download.pytorch.org/whl/cu126
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 65.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 86.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.3/201.3 MB 116.5 MB/s eta 0:00:0000:0100:01
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requ

In [2]:
!pip install https://github.com/state-spaces/mamba/releases/download/v2.3.0/mamba_ssm-2.3.0+cu12torch2.7cxx11abiFALSE-cp312-cp312-linux_x86_64.whl

  Using cached https://github.com/state-spaces/mamba/releases/download/v2.3.0/mamba_ssm-2.3.0+cu12torch2.7cxx11abiFALSE-cp312-cp312-linux_x86_64.whl (533.3 MB)


In [3]:
!git clone https://github.com/Taya-san/SKRIPSIAHMOMENT.git

Cloning into 'SKRIPSIAHMOMENT'...
remote: Enumerating objects: 141, done.
remote: Counting objects: 100% (141/141), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 141 (delta 82), reused 100 (delta 41), pack-reused 0 (from 0)
Receiving objects: 100% (141/141), 802.65 KiB | 3.17 MiB/s, done.
Resolving deltas: 100% (82/82), done.


In [1]:
%cd SKRIPSIAHMOMENT/
%ls
!git pull

/content/SKRIPSIAHMOMENT
mamba_quin/  notebooks+pdf/  README.md  utils/
Already up to date.


In [2]:
import torch
from torch import nn
import numpy as np
import transformers
from transformers import AutoModel, AutoConfig, AutoTokenizer
from datasets import load_dataset
import sklearn.model_selection
import mamba_ssm
try:
    from mamba_ssm.ops.triton.selective_state_update import selective_state_update
    from mamba_ssm.ops.selective_scan_interface import selective_scan_fn, mamba_inner_fn
    
    mamba_ssm.selective_state_update = selective_state_update
    mamba_ssm.selective_scan_fn = selective_scan_fn
    mamba_ssm.mamba_inner_fn = mamba_inner_fn

except ImportError as e:
    print(f"Patch failed! Error: {e}")

In [3]:
from mamba_quin import MambaQuin, MambaQuinConfig
from utils import train_modelnoclt

In [4]:
config = MambaQuinConfig(
    backbone_model = "state-spaces/mamba-130m-hf",
    num_classes = 2
)

model = AutoModel.from_config(config)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Loading weights:   0%|          | 0/242 [00:00<?, ?it/s]

In [5]:
dataset = load_dataset("stanfordnlp/imdb")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [6]:
tokenizer = AutoTokenizer.from_pretrained(config.backbone_model)

In [ ]:
train_modelnoclt(model, tokenizer, 3, dataset_dict=dataset, batch_size=2, tr_data=None, ts_data=None)

Epoch 1/3: 100%|██████████| 12500/12500 [31:58<00:00,  6.51it/s, loss=0.349]


Epoch 1 loss: 0.3513


Epoch 2/3:  19%|█▊        | 2326/12500 [05:57<31:07,  5.45it/s, loss=0.347]

In [25]:
!nvidia-smi

Sun Feb  8 10:11:28 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   61C    P0             29W /   70W |   15092MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----